In [1]:
import numpy as np
import tensorflow as tf
from tqdm.autonotebook import tqdm, trange
import glob

2023-06-07 16:03:11.316675: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/tmp/ipykernel_34619/1350430868.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Variables

In [2]:
data_folder = "/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/"
train_record_name = "train_ds.tfrecords"
val_record_name = "validation_ds.tfrecords"
test_record_name = "test_ds.tfrecords"

In [3]:
# Get list of file names for real components.
# Expects that there is always a matching im file
re_glob = glob.glob(data_folder + "*_re_*.bin")


print(re_glob)
file_amount = len(re_glob)
print(file_amount)

['/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_06.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_22.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_11.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_23.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_18.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_00.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_17.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_13.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_14.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_05.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_20.bin', '/app/local_home/cmorin/Tx-Id/datasets/robotRx/20230525_16h45/enreg_re_21.bin', '/app/local_home/cmorin/Tx-Id/datasets/

In [4]:
cmplx_list = []
tx_id_list = []
for file_re in re_glob:
    tx_num = int(file_re.split('/')[-1].split('_')[-1][:-4]) #Extract tx id from file name
    # rx_num = int(file_re.split('/')[-1].split('_')[-3]) #Extract rx id from file name, may break if it's not provided

    print(tx_num)

    file_im = file_re.replace("re_", "im_")
    re = np.fromfile(file_re, dtype=np.float32, count=-1)
    im = np.fromfile(file_im, dtype=np.float32, count=-1)
    cmplx = re + (1j * im)

    if len(cmplx) > 0: # Avoid tripping on empty files
        cmplx = cmplx[: (len(cmplx)//600) * 600] # Get integer number of payloads
        cmplx = np.reshape(cmplx, (-1, 600))
        cmplx_list.append(cmplx)
        tx_id_list.append(tx_num * np.ones((cmplx.shape[0], 1), dtype=np.int64))

# We have a list of complex matrices (one list element per file), with a tx_number to go with each payload
print(cmplx_list)
print(tx_id_list)

6
22
11
23
18
0
17
13
14
5
20
21
19
10
24
4
1
3
8
16
9
12
25
7
2
15
[array([[-1.52592504e-04-4.27259016e-04j, -5.79851563e-04-2.74666527e-04j,
         9.15555065e-05-6.10370014e-04j, ...,
        -1.25125854e-03-9.12503246e-03j,  1.22074003e-03-4.79140459e-03j,
         6.19525602e-03-3.08236876e-03j],
       [-9.15555065e-05+8.54518032e-04j, -5.79851563e-04-2.13629508e-04j,
        -1.83111013e-04-3.35703517e-04j, ...,
         4.48621996e-03-9.52177308e-03j,  2.96029472e-03-2.50251708e-03j,
         8.63673631e-03+8.23999580e-04j],
       [ 7.93481071e-04+3.96740536e-04j, -1.12918462e-03-1.22074009e-04j,
         9.15555065e-05+1.52592504e-04j, ...,
         1.73955457e-03-9.91851278e-03j,  1.05594015e-02-2.22785072e-03j,
         2.99081323e-03+1.40385109e-03j],
       ...,
       [ 5.18814544e-04-3.05185022e-05j,  4.27259016e-04+6.10370043e-05j,
         3.66222026e-04-9.15555065e-05j, ...,
        -9.73540172e-03-9.15555051e-04j, -3.75377573e-03-7.62962550e-03j,
         2.105776

In [5]:
concat_cmplx = np.concatenate(cmplx_list, axis=0)
concat_tx_id_list = np.concatenate(tx_id_list, axis=0)

In [6]:
base_ds = tf.data.Dataset.from_tensor_slices((concat_cmplx, concat_tx_id_list))
total_packet_num = base_ds.cardinality().numpy()
print(total_packet_num)

2023-06-07 16:03:30.306351: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10319 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:3b:00.0, compute capability: 6.1
2023-06-07 16:03:30.307679: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10319 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:b1:00.0, compute capability: 6.1
2023-06-07 16:03:30.316878: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4340942400 exceeds 10% of free system memory.
2023-06-07 16:03:36.071549: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4340942400 exceeds 10% of free system memory.


904363


In [7]:
shuffled_ds = base_ds.shuffle(buffer_size=total_packet_num)

In [8]:
train_packet_num = int(0.7 * total_packet_num)
val_packet_num = int(0.1 * total_packet_num)
test_packet_num = int(0.2 * total_packet_num)
print(f"Packet number in total {total_packet_num}, train {train_packet_num}, validation {val_packet_num}, and test {test_packet_num}")

Packet number in total 904363, train 633054, validation 90436, and test 180872


In [9]:
train_ds = shuffled_ds.take(train_packet_num)
val_ds = shuffled_ds.skip(train_packet_num).take(val_packet_num)
test_ds = shuffled_ds.skip(train_packet_num+val_packet_num)

In [14]:
# @tf.function
def example_maker(data, label):
    example = tf.train.Example(
          features=tf.train.Features(
              feature={
                  'label': tf.train.Feature(int64_list=tf.train.Int64List(value=label)),
                  'data-re': tf.train.Feature(float_list=tf.train.FloatList(value=tf.math.real(data))),
                  'data-im': tf.train.Feature(float_list=tf.train.FloatList(value=tf.math.imag(data)))
              }))
    return example

with tf.io.TFRecordWriter(data_folder+train_record_name) as writer:
    for cmplx_vec, tx_num in tqdm(train_ds, total=train_packet_num):
        example = example_maker(cmplx_vec, tx_num)
        
        writer.write(example.SerializeToString())

  0%|          | 0/633054 [00:00<?, ?it/s]

TypeError: in user code:

    File "/tmp/ipykernel_34619/2989446216.py", line 3, in example_maker  *
        example = tf.train.Example(

    TypeError: Value must be iterable
